In [71]:
# Программа для формирования базы данных текстов статей с тегами, размещенных на сайте https://globalaffairs.ru (Журнал "Россия в глобальной политике")


In [72]:
def get_text(r):
    # Извлечение текста из статьи
    #article_title - Название статьи
    #article_text - Текст статьи без ссылок
    #article_tags - метки статьи
   
    from bs4 import BeautifulSoup
    import re
    article_tags = []
    article_text = ''
    soup = BeautifulSoup(r.text, 'lxml')
    article_title = soup.find('div', {'class': 'dk-cover__title'}).text
    article_body = soup.find_all('div', {'class': 'dk-article__body'})
    tags_item = soup.find_all('a', {'class': 'dk-article__tags-item'})
    #print (article_title)
    #print (article_body)
    #print (tags_item)
    for item in article_body:
        soup_temp = BeautifulSoup(item.text, 'lxml')
        text_post = re.sub(r'https?:\/\/.*[\r\n]*', '', soup_temp.text, flags=re.MULTILINE)
        text_post = re.sub(r'\[\d+\]', ' ', text_post, flags=re.MULTILINE) #Удаление ссыдок на источники в тексте
        text_post = re.sub('\n\n', '\n', text_post, flags=re.MULTILINE)
        article_text = article_text + '\n'+ text_post
    article_text, sep, tail = article_text.partition('Сноски') # Удаление сносок из текста
    article_tags= [p.text for p in tags_item]
    return article_title, article_text, article_tags

In [73]:
def get_article(article_url):
    import requests
    from fake_useragent import UserAgent
    import random
    import time
    from bs4 import BeautifulSoup
    url = article_url
    useragent = {'User-Agent': UserAgent().random}
    try:
        r = requests.get(url, headers=useragent)
        time.sleep(random.randrange(1, 10))
        article_title, article_text, article_tags = get_text(r)
        #print(article_title)       
        #print(article_text)
        #print(article_tags)
        # Запись в файл
        try:
            with open('./RUSinGlobPolit_Dataset/{}.txt'.format(url[url.index('articles/')+9:-1]), "w", encoding="utf-8") as some_file:
                print(" ".join([article_title, article_text]), file=some_file)
                some_file.close() 
            with open('./RUSinGlobPolit_Dataset/tags/{}_tags.txt'.format(url[url.index('articles/')+9:-1]), "w", encoding="utf-8") as some_file:
                print(",".join(article_tags), file=some_file)
                some_file.close()                
        except Exception as ex:
            print(ex)
    except Exception as ex:
        print(ex)
    return url[url.index('articles/')+9:-1], " ".join([article_title, article_text]), article_tags

In [74]:
def get_url_list():
    import requests
    from fake_useragent import UserAgent
    import random
    import time
    from bs4 import BeautifulSoup
    url = 'https://globalaffairs.ru/issues/archive/'
    useragent = {'User-Agent': UserAgent().random}
    jornal_url_list = []
    article_url_list = []
    try:
        r = requests.get(url, headers=useragent)
        time.sleep(random.randrange(1, 10))
        soup = BeautifulSoup(r.text, 'lxml')
        for a in soup.find_all('a', {'class': 'dk-tile__link'} and {'class': 'link_bb_13'}):
            if a['href']!= '/subscription/':
                jornal_url_list.append(a['href'])            
        for jornal_url in jornal_url_list:
            r = requests.get(jornal_url, headers=useragent)
            time.sleep(random.randrange(1, 10))
            soup = BeautifulSoup(r.text, 'lxml')
            for a in soup.find_all('a', {'class': 'dk-toc__item'}):
                if a['href']!= '/subscription/':
                    article_url_list.append(a['href'])            
    except Exception as ex:
        print(ex)
    return article_url_list

In [75]:
article_url_list = get_url_list()
import time
import random
article_text = ''
article_tags = []
article_data = {}
try:
    for article_url in article_url_list:
        print (article_url)
        article_id, article_text, article_tags = get_article(article_url)
        print (article_id)        
        article_data [article_id] = {'article_text': article_text, 'tags': article_tags }        
        time.sleep(random.randrange(1, 10))   
    import codecs, json
    with codecs.open('RUSinGlobPolit_Dataset.json', 'w', encoding='utf-8') as f:
         f.write(json.dumps(article_data, sort_keys = True, ensure_ascii=False))
except Exception as ex:
    print(ex)

https://globalaffairs.ru/articles/ne-po-poryadku-lukyanov/
ne-po-poryadku-lukyanov
https://globalaffairs.ru/articles/rus-militarizm-czygankov/
rus-militarizm-czygankov
https://globalaffairs.ru/articles/smeshenie-vseh-fon-ranke/
smeshenie-vseh-fon-ranke
https://globalaffairs.ru/articles/ukraina-de-larosiere/
ukraina-de-larosiere
https://globalaffairs.ru/articles/modern-girinskij-azyrkin/
modern-girinskij-azyrkin
https://globalaffairs.ru/articles/miroporyadok-friman/
miroporyadok-friman
https://globalaffairs.ru/articles/neizvedannym-kursom-vong/
neizvedannym-kursom-vong
https://globalaffairs.ru/articles/bolshinstvo-bogdanov-kobrinskaya-frumkin/
bolshinstvo-bogdanov-kobrinskaya-frumkin
https://globalaffairs.ru/articles/afrikanskij-vektor-briks/
afrikanskij-vektor-briks
https://globalaffairs.ru/articles/katolichestvo-malyutin/
katolichestvo-malyutin
https://globalaffairs.ru/articles/kitajskie-otnosheniya-boroh-lomanov/
kitajskie-otnosheniya-boroh-lomanov
https://globalaffairs.ru/articles/m